In [11]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import csv
import time
import scipy.stats
#from sklearn import tree

In [12]:
#read in data
use = [1,1,1,1]
all_data = []
all_data2 = []



X_training = []
y_training = []
header = []
c = 0

with open("DataSets/Wind Power Generation Data - Forecasting/Location1.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if( c == 0):
            header = row[1:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row[1:-1]])
        y_training.append(float(row[-1]))
        #print(row)

X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])
print(len(X_training))
print(len(y_training))
all_data.append([X_train, X_test, y_train, y_test,header])


X_training = []
y_training = []
header = []
c = 0
with open("DataSets/auto+mpg/auto-mpg.data","r") as file:
    r = csv.reader(file)
    header  = ['cylinders','displacement','horsepower','weight','acceleration','model year','origin']
    for row in r:   
        line = row[0].split(' ')
        #print(line)      
        if( '?' in line):

            continue

        buff = [float(x) for x in row[0][1:].split(' ') if x!='' and x.replace('.', '', 1).isdigit() ]
        X_training.append([float(x) for x in row[0][1:].split(' ') if x!='' and x.replace('.', '', 1).isdigit() ][0:7])
        y_training.append(float(row[0][0]))
        #print(row)

X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])
print(len(X_training))
print(len(y_training))
all_data.append([X_train, X_test, y_train, y_test,header])



X_training = []
y_training = []
header = []
c = 0

with open("DataSets/Graduate Admission 2/Admission_Predict_Ver1.1.csv","r") as file:
    r = csv.reader(file)
    
    for row in r:
        if(c == 0):
            header = row[1:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row[1:-1]])
        y_training.append(float(row[-1]))


X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])
print(len(X_training))
print(len(y_training))

all_data.append([X_train, X_test, y_train, y_test,header])


#read in house price dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/house_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row])
        
with open("DataSets/house_price.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        y_training.append(float(row[0]))
  
print(len(X_training))
print(len(y_training))
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])

all_data.append([X_train, X_test, y_train, y_test,header])
print(len(all_data2))

43800
43800
392
392
500
500
10000
10000
4


In [13]:
class Node:
    def __init__(self,x,y,top,bottom,mse,mean,depth,att):
        self.x = x
        self.y = y
        self.top = top
        self.bottom = bottom
        self.sub_nodes = []
        self.mse = mse
        self.mean = mean
        self.depth = depth
        self.att = att #attribute index
        #self.predict = mean
        self.leaf = 0


In [14]:
import numpy as np
class PushTree:
    def __init__(self,x,y,step_num,ratio,limit,L=1):
        self.x = x #features
        self.y = y #labels
        
        self.step_size = 0
        self.step_num = step_num
        self.ratio = ratio
        self.leaf = 0
        #print(y)
        self.mean = np.array(y).mean()

        self.limit = limit

        self.L = L
        self.att = 0
        self.sub_nodes = []
        self.depth = []

    def mse(self,data):
        if(len(data) == 0):
            #meg kell oldani az ures adathalmazokat
            #print(np.square(np.subtract( data,data.mean() )).mean())
            #print('NaN')
            return float('NaN')
            #print(data,'Here')
        return np.square(np.subtract( data,data.mean() )).mean()

    def sort_f(self,X,y,column):

        sortf = zip(X,y)
        sortf= sorted(sortf,key = lambda x:x[0][column])
        X = np.array([x for x,y in sortf])
        y = np.array([y for x,y in sortf])
    
        return X,y
    
    def generate_mask(self,x):
        mask = []# ne index mask legyen hanem a split pontok
        #print("Max: ",str(max(x)))
        step_size = (max(x)-min(x))/self.ratio#self.ratio = len(child nodes)
        self.step_size = step_size

        for i in range(0,self.ratio+1):
            mask.append(min(x) + (i*step_size) )


        return mask
    
    def idenctical(self,a,b):
        
        for i in range(len(a)):
            if(a[i]!=b[i]):
                return 0
        return 1
    
    def ErrosSum(self,mask,vector,x,y):
        error_sum_local=0
        split_data_local=[]
        for i in range(1,len(mask)):
                    
                    split_data_local.append([x[ (mask[i-1] <= vector) & (vector<= mask[i])], y[ (mask[i-1]<=vector) & (vector<=mask[i]) ]])

                    error_sum_local += self.mse(split_data_local[-1][1])
        return error_sum_local
    
    def getPartitions(self,vector,y,mask,j,d1,d2):
        sub_att_left_rshift =   vector[(mask[j-1] <= vector) & (vector < mask[j]+d1)]#bal
        sub_lab_left_rshift =  y[(mask[j-1] <= vector) & (vector < mask[j]+d1)] 


        if(j == len(mask)-2):

            sub_att_right_rshift =  vector[(mask[j]+d1 <= vector) & (vector <= mask[j+1])]
            sub_lab_right_rshift =  y[(mask[j]+d1 <= vector) & (vector <= mask[j+1])]

            sub_att_right_lshift =   vector[(mask[j]-d2 <= vector) & (vector <= mask[j+1])]#jobb
            sub_lab_right_lshift =  y[(mask[j]-d2 <= vector) & (vector <= mask[j+1])] 

        else:
            sub_att_right_rshift =  vector[(mask[j]+d1 <= vector) & (vector < mask[j+1])]
            sub_lab_right_rshift =  y[(mask[j]+d1 <= vector) & (vector < mask[j+1])] 

            sub_att_right_lshift =   vector[(mask[j]-d2 <= vector) & (vector < mask[j+1])]#jobb
            sub_lab_right_lshift =  y[(mask[j]-d2 <= vector) & (vector < mask[j+1])] 

        sub_att_left_lshift =   vector[(mask[j-1] <= vector) & (vector < mask[j]-d2)]#bal
        sub_lab_left_lshift =  y[(mask[j-1] <= vector) & (vector < mask[j]-d2)] 
        
        return sub_lab_left_rshift, sub_lab_right_rshift, sub_lab_right_lshift,sub_lab_left_lshift

    def getLoss(self,sub_lab_left,sub_lab_right,sub_lab_left_rshift,sub_lab_right_rshift,sub_lab_right_lshift,sub_lab_left_lshift):
                #print('Right,left: ',len(sub_lab_left),len(sub_lab_right))
            h0 = len(sub_lab_left)+len(sub_lab_right)
            #h0=1
            #error = (len(sub_lab_left)/h0*self.mse(sub_lab_left) + len(sub_lab_right)/h0*self.mse(sub_lab_right) )

            sub_lab = np.concatenate((sub_lab_left,sub_lab_right),axis=0)
            error = self.mse(sub_lab)
            h = len(sub_lab_left_rshift) + len(sub_lab_right_rshift)
            h2 = len(sub_lab_right_lshift) + len(sub_lab_left_lshift)

            e1 = (len(sub_lab_right_rshift)/h*self.mse(sub_lab_right_rshift) +  len(sub_lab_left_rshift)/h*self.mse(sub_lab_left_rshift))
            e2 = (len(sub_lab_right_lshift)/h2*self.mse(sub_lab_right_lshift) +  len(sub_lab_left_lshift)/h2*self.mse(sub_lab_left_lshift))

            loss_right_shift = error -  (len(sub_lab_right_rshift)/h*self.mse(sub_lab_right_rshift) +  len(sub_lab_left_rshift)/h*self.mse(sub_lab_left_rshift)) #jobb

            loss_left_shift =  error - (len(sub_lab_right_lshift)/h2*self.mse(sub_lab_right_lshift) +  len(sub_lab_left_lshift)/h2*self.mse(sub_lab_left_lshift)) #bal

            return loss_right_shift,loss_left_shift,e1,e2

    def best_split(self,x,y,depth):


        split_data = []
        min_error = 99999999999999999999999999
        re_data = []
        re_mask = 0
        att = -1
        x = np.array(x)
        y = np.array(y)

        coeff = 0
        att_c = 0
        for k in range(len(x[0])-0):
           

            x,y = self.sort_f(x,y,k)
            split_vector = []
            split_data = []
            vector = x[:,k] # get an attribute
            
            mask = self.generate_mask(vector)
            error_sum = 0
            if(mask.count(mask[0]) == len(mask)):
                continue

            length = len(mask)
            optimal = [0 for x in range(self.ratio-1)]
            local_min_error = 9999999999999999999999999
            
            pre_masks = []
            
            while(0 in optimal):
                           
                sub_attributes = [  vector[(mask[i] < vector) & (vector<=mask[i+1])] for i in range(0,len(mask)-1)] 
                sub_labels = [ y[(mask[i]<vector) & (vector<=mask[i+1])] for i in range(0,len(mask)-1)]
                error = 0
                split_data_local = []
                error_sum_local = 0

                j = 1
                
                while(j<=length-2):
                    error_sum_local_left = 0
                    error_sum_local_right = 0
                    
                    if(optimal[j-1] == 1):
                        j+=1
                        continue
                    
                    d1  =  abs(max(vector)-min(vector))/200
                    d2  =  abs(max(vector)-min(vector))/200
                    #constraints check
                    if(mask[j]+d1>=mask[j+1]):
                        d1 = 0
                    if(mask[j]-d2<=mask[j-1]):
                        d2 = 0


                    sub_lab_left = y[(mask[j-1] <= vector) & (vector < mask[j])] 

                    if(j+1 == len(mask)-1):
                        sub_lab_right = y[(mask[j] <= vector) & (vector <= mask[j+1])]
                    else:
                        sub_lab_right = y[(mask[j] <= vector) & (vector < mask[j+1])]
                    
                    if( (len(sub_lab_left) == 0 or len(sub_lab_right) == 0) and len(mask)>3 and j!=0 and j!=length-1):
                        
                        mask.remove(mask[j])
                       
                        length -= 1
                        if(j>1):
                            j-=1
                        sub_lab_left = y[(mask[j] <= vector) & (vector < mask[j+1])] 
                        sub_lab_right = y[(mask[j-1] <= vector) & (vector <= mask[j])]


                    #get partitions for the modified split point(both)
                    sub_lab_left_rshift, sub_lab_right_rshift, sub_lab_right_lshift,sub_lab_left_lshift = self.getPartitions(vector,y,mask,j,d1,d2)
                    
                    loss_right_shift,loss_left_shift,e1,e2 = self.getLoss(sub_lab_left,sub_lab_right,
                                                                    sub_lab_left_rshift,
                                                                    sub_lab_right_rshift,
                                                                    sub_lab_right_lshift,
                                                                    sub_lab_left_lshift)

                   
                    if( loss_right_shift>loss_left_shift and error>e1 and mask[j]+d1<mask[j+1]):
                        mask[j]+=d1 
                
                    elif(loss_right_shift<loss_left_shift and error>e2 and mask[j]-d2>mask[j-1] ):
                        mask[j]-=d2
                        
                    else:
                        optimal[j-1] = 1
                    j+=1

                
                
                if(len(pre_masks)<2):
                    pre_masks.append(mask.copy())
                    continue
                #if all split points are optimal we stop splitting. 
                #In case it end up in a loop we choose the one whic has lower error rate 
                if(self.idenctical(pre_masks[-2],mask)):
                        #print(pre_masks,mask)
                        err1 = self.ErrosSum(mask,vector,x,y)
                        err2 = self.ErrosSum(pre_masks[-1],vector,x,y)
                        if(err1<=err2):
                            #print("break")
                            for i in range(len(optimal)):
                                optimal[i] = 1
                            break
                        else:
                            mask = pre_masks[-1].copy()                          
                            break

                pre_masks.append(mask.copy())

            #calculate error    
            for i in range(1,len(mask)):
                split_vector.append([vector[ (mask[i-1] <= vector) & (vector<=mask[i]) ], y[ (mask[i-1]<=vector) &  (vector<=mask[i] )] ])
                split_data.append([x[ (mask[i-1] <= vector) & (vector<= mask[i])], y[ (mask[i-1]<=vector) & (vector<=mask[i]) ]])

                error_sum += self.mse(split_data[-1][1])

            if(error_sum<min_error):
                min_error = error_sum
                re_data = split_data 
                re_mask = mask
                att = k

        return re_data,att,re_mask
   

    def fit(self,x,y,depth=0,root=0):

        split_data,att,mask = self.best_split(x,y,depth)

        sub_nodes = []
        s = 0
        for i,node in enumerate(split_data):

            sub_node = Node(
                x = np.array(node[0]),
                y = np.array(node[1]),
                top = mask[i+1],
                bottom = mask[i],
                mse = np.square(np.subtract( np.array(node[1]),np.array(node[1]).mean() )).mean(),
                mean = np.array(node[1]).mean(),
                depth = depth+1,
                att = att
            )

            s += len(node[0])

            sub_nodes.append(sub_node)
 
        for node in sub_nodes:

            if(len(node.x)<self.limit or self.mse(node.y)<=0.0):#slef.limit
   
                self.depth.append(depth+1)
                node.leaf = 1
            else: 
                #print(depth)
                node.sub_nodes = self.fit(node.x,node.y,depth+1)
        if(root):
            self.sub_nodes = sub_nodes
            self.att = att
        else:
            return sub_nodes
        

    def predict(self,X,node, root=0):

        if(root == 0 and node.leaf == 1):

            return node.mean
        else:
            found = False
            for child in node.sub_nodes:

                if(X[child.att] >= child.bottom and X[child.att] <= child.top):
                   
                    return self.predict(X,child)

            min_dist = 999999999999999999999
            min_child = 0
            for child in node.sub_nodes:
                dist1 = abs(X[child.att]-child.bottom)
                dist2 = abs(X[child.att]-child.top)
                if(min(dist1,dist2)<min_dist):
                    min_dist = min(dist1,dist2)
                    min_child = child
                
                    
            return self.predict(X,min_child)
        return node.mean

In [ ]:
forest = []
times = []
depth = []
limits = [120,115,110,105,100,95,90,85,80,75,70,65,60,55,50,45,40,35,30,25,20,15,10]



for i in all_data:

    f = []
    t = []
    d = []
    for l in limits:
        
        print("-------------------------------------------------------")
    
        start = time.time()
        # create a regressor object
        tree = PushTree(i[0],i[2],10,3,l)#irirs 30,3;10,3 blob 20,3;10,5;30,5
        PushTree.fit(tree,i[0],i[2],root=1)
    
        end = time.time()
        f.append(tree)
        t.append(end-start)
        d.append(tree.depth)
        
        
    forest.append(f)
    times.append(t)
    depth.append(d)

In [ ]:
predicted_value = []#predicted points
difference = []
MSE = []
sum_y = []

all_mse = []
print(len(forest))
#for each tree we calculate the mean squared error
#all_data = [X_train,X_test,y_train,y_test,header]
for v in range(len(forest)):
    MSE = []
    print(len(forest[v]))
    
    for i in forest[v]:
        z=v
        j = all_data[z]
        
        X_test = j[1]
        y_test = j[3]
        pv = []
        error = 0
        mse = 0
        for k in range(len(X_test)): 

            p = [i.predict(X_test[k],i,root=1)]
            error += abs(y_test[k]-p[0])
            mse += (y_test[k]-p[0])**2
            pv.append(p)

        sum_y.append(sum(y_test))
        predicted_value.append(pv)
        difference.append(error)
        MSE.append(mse/len(X_test))

    all_mse.append(MSE)

 
print(all_mse)
print(time)



4
23
23
23
23
[[0.02724781073005635, 0.027287221804721477, 0.02742428447945334, 0.02743183400834298, 0.02743256780237182, 0.027450263681262237, 0.027497500248895506, 0.027572761472466536, 0.02755601692745722, 0.027542607627212626, 0.02752067040916898, 0.02755371821116709, 0.02764422212960503, 0.027867245313676615, 0.02795988145139888, 0.02838369466982908, 0.028567994267017914, 0.02912933192726963, 0.029366220879336276, 0.0300203646535265, 0.03125212238921357, 0.03353672887254822, 0.036966018079506995], [0.2740196067422623, 0.2740196067422623, 0.2740196067422623, 0.2740196067422623, 0.2740196067422623, 0.2446886745871816, 0.2446886745871816, 0.24062537701597783, 0.24062537701597783, 0.24062537701597783, 0.24062537701597783, 0.24032519600939664, 0.24032519600939664, 0.24032519600939664, 0.21203520043807308, 0.21203520043807308, 0.2126334102888383, 0.20769387507135204, 0.19981125510086786, 0.19981125510086786, 0.20556315101152173, 0.19500041767766663, 0.1940540215148413], [0.0040200720644

In [ ]:
with open("Results3/test5/difference_push3_mse_nbrt2.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for row in all_mse:
        writer.writerow(row)
        
    #for row in all_mse:
    #    writer.writerow(row)
with open("Results3/test5/time_push3_mse_nbrt2.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    for row in times:
        
        writer.writerow(row)
    writer.writerow(" ")
with open("Results3/test5/depth_push3_mse_nbrt2.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    for row in depth:
        
        writer.writerow(row)
    writer.writerow(" ")

#[[0.33411703274340626]]   
print(all_mse)
print(time)
#[[0.027498811208953384, 0.02753238427915346, 0.027665468882463143, 0.027652339637128837, 0.027643725306100132, 0.02765934094996693, 0.027705946902967824, 0.027794047749282298, 0.027822641795534654, 0.027826734526930446, 0.02801261434023049, 0.028117659940493832, 0.028439375503341566, 0.028878437342339667, 0.02927564594020998, 0.029767212947846157, 0.03024093912483305, 0.03154949298987045, 0.033221682330473665, 0.03519716669447423, 0.038913567324389046], [1.0996710893507982, 1.0996710893507982, 1.0996710893507982, 1.0822163323190166, 1.0822163323190166, 1.1005404530940481, 1.1005404530940481, 1.1005404530940481, 1.0705321443563909, 1.0705321443563909, 1.0705321443563909, 1.0705321443563909, 1.0649861586352236, 0.9784475028630318, 0.9791034438311904, 0.9785140122241914, 0.9652530205381754, 0.9669545784014678, 0.9669545784014678, 0.9333895505122384, 0.9290153356232318], [0.006636593463016858, 0.006636593463016858, 0.006636593463016858, 0.006636593463016858, 0.006636593463016858, 0.005596435039194314, 0.005596435039194314, 0.005596435039194314, 0.005596435039194314, 0.005495020098211673, 0.005495020098211673, 0.005468853562112378, 0.005468853562112378, 0.005468853562112378, 0.005416744788925971, 0.005686292518208462, 0.005686292518208462, 0.005954475166069271, 0.006124208058485495, 0.006179051483318975, 0.006625014376417231], [30.551485258527514, 28.290360590083505, 27.71796757005346, 27.502211766592072, 26.730694160702182, 26.046406317479875, 25.139155379905333, 24.577521296184376, 24.075297418236552, 23.26188118250014, 22.170710043191118, 21.711716822488142, 21.316840035621926, 20.362845721280035, 18.975866683468244, 17.878797666235304, 16.529293891070292, 15.616361119823456, 14.24365128861582, 12.448764403192392, 10.634810545877546]]
#[[0.027498811208953384, 0.02753238427915346, 0.027665468882463143, 0.027652339637128837, 0.027643725306100132, 0.02765934094996693, 0.027705946902967824, 0.027794047749282298, 0.027822641795534654, 0.027826734526930446, 0.02801261434023049, 0.028117659940493832, 0.028439375503341566, 0.028878437342339667, 0.02927564594020998, 0.029767212947846157, 0.03024093912483305, 0.03154949298987045, 0.033221682330473665, 0.03519716669447423, 0.038913567324389046], [1.0996710893507982, 1.0996710893507982, 1.0996710893507982, 1.0822163323190166, 1.0822163323190166, 1.1005404530940481, 1.1005404530940481, 1.1005404530940481, 1.0705321443563909, 1.0705321443563909, 1.0705321443563909, 1.0705321443563909, 1.0649861586352236, 0.9784475028630318, 0.9791034438311904, 0.9785140122241914, 0.9652530205381754, 0.9669545784014678, 0.9669545784014678, 0.9333895505122384, 0.9290153356232318], [0.006636593463016858, 0.006636593463016858, 0.006636593463016858, 0.006636593463016858, 0.006636593463016858, 0.005596435039194314, 0.005596435039194314, 0.005596435039194314, 0.005596435039194314, 0.005495020098211673, 0.005495020098211673, 0.005468853562112378, 0.005468853562112378, 0.005468853562112378, 0.005416744788925971, 0.005686292518208462, 0.005686292518208462, 0.005954475166069271, 0.006124208058485495, 0.006179051483318975, 0.006625014376417231], [30.551485258527514, 28.290360590083505, 27.71796757005346, 27.502211766592072, 26.730694160702182, 26.046406317479875, 25.139155379905333, 24.577521296184376, 24.075297418236552, 23.26188118250014, 22.170710043191118, 21.711716822488142, 21.316840035621926, 20.362845721280035, 18.975866683468244, 17.878797666235304, 16.529293891070292, 15.616361119823456, 14.24365128861582, 12.448764403192392, 10.634810545877546]]


[[0.02724781073005635, 0.027287221804721477, 0.02742428447945334, 0.02743183400834298, 0.02743256780237182, 0.027450263681262237, 0.027497500248895506, 0.027572761472466536, 0.02755601692745722, 0.027542607627212626, 0.02752067040916898, 0.02755371821116709, 0.02764422212960503, 0.027867245313676615, 0.02795988145139888, 0.02838369466982908, 0.028567994267017914, 0.02912933192726963, 0.029366220879336276, 0.0300203646535265, 0.03125212238921357, 0.03353672887254822, 0.036966018079506995], [0.2740196067422623, 0.2740196067422623, 0.2740196067422623, 0.2740196067422623, 0.2740196067422623, 0.2446886745871816, 0.2446886745871816, 0.24062537701597783, 0.24062537701597783, 0.24062537701597783, 0.24062537701597783, 0.24032519600939664, 0.24032519600939664, 0.24032519600939664, 0.21203520043807308, 0.21203520043807308, 0.2126334102888383, 0.20769387507135204, 0.19981125510086786, 0.19981125510086786, 0.20556315101152173, 0.19500041767766663, 0.1940540215148413], [0.004020072064457058, 0.00402